# Start with your own case 

In addition to the rich collcetion of **datasets**, **models** and **evaluation metrics**, **FederatedScope** also allows to create your own or introduce more to our package.

We provide `register` function to help build your own federated learning workflow.  This introduction will help you to start with your own case:

1. [Load a dataset](#data)
2. [Build a model](#model) 
3. [Create a trainer](#trainer)
4. [Introduce more evaluation metrics](#metric)

## <span id="data">1. Load a dataset</span>

We provide a function federatedscope.register.register_data to make your dataset available with three steps:

* Step1: set up your data in the following format (standalone):
    
    **Note**: This function returns a `dict`, where the `key` is the client's id, and the `value` is the data `dict` of each client with 'train', 'test' or 'val'.  You can also modify the config here.

    We take `torchvision.datasets.MNIST`, which is split and assigned to two clients, as an example:

In [ ]:
def load_my_data(config):
    import numpy as np
    from torchvision import transforms
    from torchvision.datasets import MNIST
    from torch.utils.data import DataLoader

    # Build data
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.1307], std=[0.3081])
    ])
    data_train = MNIST(root=config.data.root, train=True, transform=transform, download=True)
    data_test = MNIST(root=config.data.root, train=False, transform=transform, download=True)

    # Split data into dict
    data_dict = dict()
    train_per_client = len(data_train) // config.federate.client_num
    test_per_client = len(data_test) // config.federate.client_num

    for client_idx in range(1, config.federate.client_num + 1):
        dataloader_dict = {
            'train':
            DataLoader([
                data_train[i]
                for i in range((client_idx - 1) *
                               train_per_client, client_idx * train_per_client)
            ],
                       config.data.batch_size,
                       shuffle=config.data.shuffle),
            'test':
            DataLoader([
                data_test[i]
                for i in range((client_idx - 1) * test_per_client, client_idx *
                               test_per_client)
            ],
                       config.data.batch_size,
                       shuffle=False)
        }
        data_dict[client_idx] = dataloader_dict

    return data_dict, config

* Step2: register your data with a keyword, such as `"mydata"`

In [ ]:
from federatedscope.register import register_data

def call_my_data(config, client_cfgs=None):
    if config.data.type == "mycvdata":
        data, modified_config = load_my_data(config)
        return data, modified_config

register_data("mycvdata", call_my_data)

## <span id="model">2. Build a model</span>
We provide a function `federatedscope.register.register_model` to make your model available with three steps: (we take `ConvNet2` as an example)

* Step1: build your model with Pytorch or Tensorflow and instantiate your model class with config and data.

In [ ]:
import torch


class MyNet(torch.nn.Module):
    def __init__(self,
                 in_channels,
                 h=32,
                 w=32,
                 hidden=2048,
                 class_num=10,
                 use_bn=True):
        super(MyNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels, 32, 5, padding=2)
        self.conv2 = torch.nn.Conv2d(32, 64, 5, padding=2)
        self.fc1 = torch.nn.Linear((h // 2 // 2) * (w // 2 // 2) * 64, hidden)
        self.fc2 = torch.nn.Linear(hidden, class_num)
        self.relu = torch.nn.ReLU(inplace=True)
        self.maxpool = torch.nn.MaxPool2d(2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(self.relu(x))
        x = self.conv2(x)
        x = self.maxpool(self.relu(x))
        x = torch.nn.Flatten()(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


def load_my_net(model_config, data_shape):
    # You can also build models without local_data
    model = MyNet(in_channels=data_shape[1],
                  h=data_shape[2],
                  w=data_shape[3],
                  hidden=model_config.hidden,
                  class_num=model_config.out_channels)
    return model

* Step2: register your model with a keyword, such as `"mynet"`

In [ ]:
from federatedscope.register import register_model

def call_my_net(model_config, data_shape):
    if model_config.type == "mycnn":
        model = load_my_net(model_config, data_shape)
        return model

register_model("mycnn", call_my_net)

## <span id="trainer">3. Create a trainer</span>

FederatedScope decouples the local learning process and details of FL communication and schedule, allowing users to freely customize the local learning algorithms via the `Trainer`. We recommend user build trainer by inheriting `federatedscope.core.trainers.trainer.GeneralTorchTrainer`, for more details, please see [Trainer](https://federatedscope.io/docs/trainer/). Similarly, we provide `federatedscope.register.register_trainer` to make your customized trainer available:

* Step1: build your trainer by inheriting `GeneralTrainer`. Our `GeneralTrainer` already supports many different usages, for the advanced user, please see [federatedscope.core.trainers.trainer.GeneralTrainer]() for more details.

In [ ]:
from federatedscope.core.trainers import GeneralTorchTrainer

class MyTrainer(GeneralTorchTrainer):
    pass

* Step2: register your trainer with a keyword, such as `"mytrainer"`

In [ ]:
from federatedscope.register import register_trainer

def call_my_trainer(trainer_type):
    if trainer_type == 'mycvtrainer':
        trainer_builder = MyTrainer
        return trainer_builder

register_trainer('mycvtrainer', call_my_trainer)

## <span id="metric">4. Introduce more evaluation metrics</span>
We provide a number of metrics to monitor the entire federal learning process. You just need to list the name of the metric you want in `cfg.eval.metrics`. We currently support metrics such as loss, accuracy, etc. (See [federatedscope.core.evaluator](federatedscope/core/evaluator.py) for more details).

We also provide a function `federatedscope.register.register_metric` to make your evaluation metrics available with three steps:

* Step1: build your metric (see [federatedscope.core.context](federatedscope/core/context.py) for more about `ctx`)

In [ ]:
def cal_my_metric(ctx, **kwargs):
    return ctx["num_train_data"]

* Step2: register your metric with a keyword, such as `"mymetric"`

In [ ]:
from federatedscope.register import register_metric

def call_my_metric(types):
    if "mymetric" in types:
        metric_builder = cal_my_metric
        return "mymetric", metric_builder

register_metric("mymetric", call_my_metric)

## Let's start!
* Set your data, model, trainer and metric first.

In [ ]:
from federatedscope.core.configs.config import global_cfg

cfg = global_cfg.clone()

cfg.data.type = 'mycvdata'
cfg.data.root = 'data'
cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.1307], 'std': [0.3081]}]]
cfg.model.type = 'mycnn'
cfg.model.out_channels = 10
cfg.trainer.type = 'mycvtrainer'
cfg.eval.metric = ['mymetric']

* Configure other options in `cfg`.

In [ ]:
cfg.use_gpu = False
cfg.best_res_update_round_wise_key = "test_loss"

cfg.federate.mode = 'standalone'
cfg.federate.local_update_steps = 5
cfg.federate.total_round_num = 20
cfg.federate.sample_client_num = 5
cfg.federate.client_num = 5

cfg.train.optimizer.lr = 0.001
cfg.train.optimizer.weight_decay = 0.0
cfg.grad.grad_clip = 5.0

cfg.criterion.type = 'CrossEntropyLoss'
cfg.seed = 123
cfg.eval.best_res_update_round_wise_key = "test_loss"

* Start your FL process!

In [ ]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed
from federatedscope.core.auxiliaries.logging import update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)
Fed_runner = FedRunner(data=data,
                       server_class=get_server_cls(cfg),
                       client_class=get_client_cls(cfg),
                       config=cfg.clone())
Fed_runner.run()